PWC-Net-small model finetuning (with cyclical learning rate schedule)
=======================================================

In this notebook we:
- Use a small model (no dense or residual connections), 6 level pyramid, uspample level 2 by 4 as the final flow prediction
- Train the PWC-Net-small model on a mix of the `FlyingChairs` and `FlyingThings3DHalfRes` dataset using a Cyclic<sub>short</sub> schedule of our own
- Let the Cyclic<sub>short</sub> schedule oscillate between `2e-05` and `1e-06` for 200,000 steps
- Switch to the "robust" loss described in the paper, instead of the "multiscale" loss used during training

Below, look for `TODO` references and customize this notebook based on your own needs.

## Reference

[2018a]<a name="2018a"></a> Sun et al. 2018. PWC-Net: CNNs for Optical Flow Using Pyramid, Warping, and Cost Volume. [[arXiv]](https://arxiv.org/abs/1709.02371) [[web]](http://research.nvidia.com/publication/2018-02_PWC-Net%3A-CNNs-for) [[PyTorch (Official)]](https://github.com/NVlabs/PWC-Net/tree/master/PyTorch) [[Caffe (Official)]](https://github.com/NVlabs/PWC-Net/tree/master/Caffe)

In [2]:
"""
pwcnet_finetune.ipynb

PWC-Net model finetuning.

Written by Phil Ferriere

Licensed under the MIT License (see LICENSE for details)

Tensorboard:
    [win] tensorboard --logdir=E:\\repos\\tf-optflow\\tfoptflow\\pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned
    [ubu] tensorboard --logdir=/media/EDrive/repos/tf-optflow/tfoptflow/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned
"""
from __future__ import absolute_import, division, print_function
import sys
from copy import deepcopy
from dataset_mpisintel import MPISintelDataset
from dataset_base import _DEFAULT_DS_TUNE_OPTIONS
from dataset_flyingchairs import FlyingChairsDataset
from dataset_flyingthings3d import FlyingThings3DHalfResDataset
from dataset_mixer import MixedDataset
from model_pwcnet import ModelPWCNet, _DEFAULT_PWCNET_FINETUNE_OPTIONS

In [1]:
!nvidia-smi

Wed Apr 24 12:22:50 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.40.04    Driver Version: 418.40.04    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           Off  | 00000000:04:00.0 Off |                  Off |
| N/A   35C    P0    50W / 250W |    749MiB / 24451MiB |     19%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P40           Off  | 00000000:06:00.0 Off |                  Off |
| N/A   34C    P0    50W / 250W |    749MiB / 24451MiB |     20%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

## TODO: Set this first!

In [3]:
# TODO: You MUST set dataset_root to the correct path on your machine!
if sys.platform.startswith("win"):
    _DATASET_ROOT = 'E:/datasets/'
else:
    _DATASET_ROOT = '/Vol1/dbstore/datasets/sintel/'
_MPISINTEL_ROOT = _DATASET_ROOT + 'Sintel_color'
    
# TODO: You MUST adjust the settings below based on the number of GPU(s) used for training
# Set controller device and devices
# A one-gpu setup would be something like controller='/device:GPU:0' and gpu_devices=['/device:GPU:0']
# Here, we use a dual-GPU setup, as shown below
# gpu_devices = ['/device:GPU:0', '/device:GPU:1']
# controller = '/device:CPU:0'
gpu_devices = ['/device:GPU:0']
controller = '/device:GPU:0'

# TODO: You MUST adjust this setting below based on the amount of memory on your GPU(s)
# Batch size
batch_size = 8

# Finetune on `sintel` mix

## Load the dataset

In [4]:
# TODO: You MUST set the batch size based on the capabilities of your GPU(s) 
#  Load train dataset
ds_opts = deepcopy(_DEFAULT_DS_TUNE_OPTIONS)
ds_opts['in_memory'] = False                          # Too many samples to keep in memory at once, so don't preload them
ds_opts['aug_type'] = 'heavy'                         # Apply all supported augmentations
ds_opts['batch_size'] = batch_size * len(gpu_devices) # Use a multiple of 8; here, 16 for dual-GPU mode (Titan X & 1080 Ti)
#ds_opts['crop_preproc'] = (256, 448)                  # Crop to a smaller input size
#ds1 = FlyingChairsDataset(mode='train_with_val', ds_root=_FLYINGCHAIRS_ROOT, options=ds_opts)
ds_opts['type'] = 'into_future'
#ds2 = FlyingThings3DHalfResDataset(mode='train_with_val', ds_root=_FLYINGTHINGS3DHALFRES_ROOT, options=ds_opts)
#ds = MixedDataset(mode='train_with_val', datasets=[ds1, ds2], options=ds_opts)

In [5]:
ds_opts = deepcopy(_DEFAULT_DS_TUNE_OPTIONS)
ds_opts['type'] = 'clean'
ds = MPISintelDataset(mode='train_with_val', ds_root=_MPISINTEL_ROOT, options=ds_opts)

In [6]:
# Display dataset configuration
ds.print_config()


Dataset Configuration:
  verbose              False
  in_memory            False
  crop_preproc         (384, 768)
  scale_preproc        None
  type                 clean
  tb_test_imgs         False
  random_seed          1969
  val_split            0.03
  aug_type             heavy
  aug_labels           True
  fliplr               0.5
  flipud               0.5
  translate            (0.5, 0.05)
  scale                (0.5, 0.05)
  mode                 train_with_val
  train size           1009
  val size             32


In [ ]:
ckpt_path = './models/pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-595000'

## Configure the finetuning

In [14]:
# Start from the default options
nn_opts = deepcopy(_DEFAULT_PWCNET_FINETUNE_OPTIONS)
nn_opts['verbose'] = True
nn_opts['ckpt_path'] = './models/pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-595000'
nn_opts['ckpt_dir'] = './pwcnet-lg-6-2-cyclic-sintel_finetuned/'
nn_opts['batch_size'] = 8
#nn_opts['x_shape'] = [2, ds_opts['crop_preproc'][0], ds_opts['crop_preproc'][1], 3]
#nn_opts['y_shape'] = [ds_opts['crop_preproc'][0], ds_opts['crop_preproc'][1], 2]
nn_opts['use_tf_data'] = True # Use tf.data reader
nn_opts['gpu_devices'] = gpu_devices
nn_opts['controller'] = controller

# Use the PWC-Net-small model in quarter-resolution mode
nn_opts['use_dense_cx'] = True
nn_opts['use_res_cx'] = True
nn_opts['pyr_lvls'] = 6
nn_opts['flow_pred_lvl'] = 2

# Robust loss as described doesn't work, so try the following:
nn_opts['loss_fn'] = 'loss_multiscale' # 'loss_multiscale' # 'loss_robust' # 'loss_robust'
nn_opts['q'] = 1. # 0.4 # 1. # 0.4 # 1.
nn_opts['epsilon'] = 0. # 0.01 # 0. # 0.01 # 0.

In [15]:
# Set the learning rate schedule. This schedule is for a single GPU using a batch size of 8.
# Below,we adjust the schedule to the size of the batch and the number of GPUs.
nn_opts['lr_policy'] = 'multisteps'
nn_opts['init_lr'] = 1e-05
nn_opts['lr_boundaries'] = [80000, 120000, 160000, 200000]
nn_opts['lr_values'] = [1e-05, 5e-06, 2.5e-06, 1.25e-06, 6.25e-07]
nn_opts['max_steps'] = 200000

# Below,we adjust the schedule to the size of the batch and our number of GPUs (2).
nn_opts['max_steps'] = int(nn_opts['max_steps'] * 8 / 8)
nn_opts['cyclic_lr_stepsize'] = int(nn_opts['cyclic_lr_stepsize'] * 8 / 8)

In [16]:
# Instantiate the model and display the model configuration
nn = ModelPWCNet(mode='train_with_val', options=nn_opts, dataset=ds)
nn.print_config()

Building model...
... model built.
Configuring training ops...
... training ops configured.
Initializing from pre-trained model at ./models/pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-595000 for finetuning...

INFO:tensorflow:Restoring parameters from ./models/pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-595000
... model initialized

Model Configuration:
  verbose                True
  ckpt_path              ./models/pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-595000
  ckpt_dir               ./pwcnet-lg-6-2-cyclic-sintel_finetuned/
  max_to_keep            10
  x_dtype                <dtype: 'float32'>
  x_shape                [2, 384, 768, 3]
  y_dtype                <dtype: 'float32'>
  y_shape                [384, 768, 2]
  train_mode             fine-tune
  adapt_info             None
  sparse_gt_flow         False
  display_step           100
  snapshot_step          1000
  val_step               1000
  val_batch_size         -1
  tb_val_imgs            t

## Finetune the model

In [17]:
# Train the model
nn.train()

Start finetuning...
Instructions for updating:
Use `tf.data.experimental.shuffle_and_repeat(...)`.
Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
2019-04-24 12:31:08 Iter 100 [Train]: loss=109.68, epe=2.80, lr=0.000010, samples/sec=6.4, sec/step=1.251, eta=2 days, 21:26:36
2019-04-24 12:32:54 Iter 200 [Train]: loss=107.05, epe=2.73, lr=0.000010, samples/sec=7.7, sec/step=1.035, eta=2 days, 9:25:57
2019-04-24 12:34:40 Iter 300 [Train]: loss=104.22, epe=2.64, lr=0.000010, samples/sec=7.8, sec/step=1.030, eta=2 days, 9:07:48
2019-04-24 12:36:26 Iter 400 [Train]: loss=100.05, epe=2.51, lr=0.000010, samples/sec=7.7, sec/step=1.033, eta=2 days, 9:17:26
2019-04-24 12:38:11 Iter 500 [Train]: loss=94.04, epe=2.34, lr=0.000010, samples/sec=7.8, sec/step=1.027, eta=2 days, 8:55:02
2019-04-24 12:39:55 Iter 600 [Train]: loss=104.23, epe=2.62, lr=0.000010, samples/sec=7.8, sec/step=1.021, eta=2 days, 8:33:57
2019-04-24 12:41:41 Iter 700 [Train]: loss=96.46, epe=2.40, lr=0.

... model saved in ./pwcnet-lg-6-2-cyclic-sintel_finetuned/pwcnet.ckpt-5000
2019-04-24 13:59:47 Iter 5100 [Train]: loss=81.31, epe=1.85, lr=0.000010, samples/sec=7.8, sec/step=1.025, eta=2 days, 7:29:18
2019-04-24 14:01:32 Iter 5200 [Train]: loss=77.48, epe=1.80, lr=0.000010, samples/sec=7.8, sec/step=1.030, eta=2 days, 7:43:29
2019-04-24 14:03:17 Iter 5300 [Train]: loss=85.32, epe=2.03, lr=0.000010, samples/sec=7.8, sec/step=1.020, eta=2 days, 7:09:30
2019-04-24 14:05:01 Iter 5400 [Train]: loss=76.71, epe=1.77, lr=0.000010, samples/sec=7.8, sec/step=1.021, eta=2 days, 7:12:44
2019-04-24 14:06:47 Iter 5500 [Train]: loss=81.73, epe=1.90, lr=0.000010, samples/sec=7.8, sec/step=1.028, eta=2 days, 7:33:49
2019-04-24 14:08:32 Iter 5600 [Train]: loss=78.86, epe=1.82, lr=0.000010, samples/sec=7.8, sec/step=1.026, eta=2 days, 7:22:54
2019-04-24 14:10:16 Iter 5700 [Train]: loss=82.06, epe=1.93, lr=0.000010, samples/sec=7.8, sec/step=1.024, eta=2 days, 7:17:22
2019-04-24 14:12:01 Iter 5800 [Trai

2019-04-24 15:33:29 Iter 10400 [Train]: loss=75.36, epe=1.69, lr=0.000010, samples/sec=7.8, sec/step=1.025, eta=2 days, 5:58:41
2019-04-24 15:35:14 Iter 10500 [Train]: loss=67.52, epe=1.47, lr=0.000010, samples/sec=7.8, sec/step=1.020, eta=2 days, 5:40:50
2019-04-24 15:36:59 Iter 10600 [Train]: loss=76.81, epe=1.73, lr=0.000010, samples/sec=7.8, sec/step=1.021, eta=2 days, 5:43:23
2019-04-24 15:38:44 Iter 10700 [Train]: loss=74.37, epe=1.65, lr=0.000010, samples/sec=7.9, sec/step=1.016, eta=2 days, 5:25:27
2019-04-24 15:40:29 Iter 10800 [Train]: loss=76.59, epe=1.79, lr=0.000010, samples/sec=7.9, sec/step=1.018, eta=2 days, 5:30:32
2019-04-24 15:42:15 Iter 10900 [Train]: loss=73.96, epe=1.66, lr=0.000010, samples/sec=7.8, sec/step=1.024, eta=2 days, 5:48:47
2019-04-24 15:44:01 Iter 11000 [Train]: loss=66.58, epe=1.46, lr=0.000010, samples/sec=7.8, sec/step=1.021, eta=2 days, 5:37:27
2019-04-24 15:44:02 Iter 11000 [Val]: loss=77.45, epe=1.94
Saving model...
INFO:tensorflow:./pwcnet-lg-6

2019-04-24 17:07:29 Iter 15700 [Train]: loss=73.98, epe=1.65, lr=0.000010, samples/sec=7.8, sec/step=1.027, eta=2 days, 4:35:35
2019-04-24 17:09:14 Iter 15800 [Train]: loss=71.07, epe=1.59, lr=0.000010, samples/sec=7.8, sec/step=1.022, eta=2 days, 4:16:49
2019-04-24 17:11:00 Iter 15900 [Train]: loss=65.59, epe=1.44, lr=0.000010, samples/sec=7.8, sec/step=1.021, eta=2 days, 4:11:31
2019-04-24 17:12:46 Iter 16000 [Train]: loss=67.66, epe=1.48, lr=0.000010, samples/sec=7.8, sec/step=1.029, eta=2 days, 4:36:37
2019-04-24 17:12:47 Iter 16000 [Val]: loss=62.00, epe=1.41
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-cyclic-sintel_finetuned/pwcnet.ckpt-16000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-cyclic-sintel_finetuned/pwcnet.ckpt-16000
2019-04-24 17:14:42 Iter 16100 [Train]: loss=69.21, epe=1.51, lr=0.000010, samples/sec=7.8, sec/step=1.027, eta=2 days, 4:27:09
2019-04-24 17:16:27 Iter 16200 [Train]: loss=64.78, epe=1.41, lr=0.000010, s

2019-04-24 18:41:36 Iter 21000 [Train]: loss=61.93, epe=1.32, lr=0.000010, samples/sec=7.8, sec/step=1.026, eta=2 days, 2:59:45
2019-04-24 18:41:37 Iter 21000 [Val]: loss=65.14, epe=1.52
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-cyclic-sintel_finetuned/pwcnet.ckpt-21000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-cyclic-sintel_finetuned/pwcnet.ckpt-21000
2019-04-24 18:43:32 Iter 21100 [Train]: loss=70.19, epe=1.54, lr=0.000010, samples/sec=7.8, sec/step=1.029, eta=2 days, 3:07:22
2019-04-24 18:45:17 Iter 21200 [Train]: loss=65.48, epe=1.42, lr=0.000010, samples/sec=7.8, sec/step=1.022, eta=2 days, 2:45:50
2019-04-24 18:47:03 Iter 21300 [Train]: loss=66.04, epe=1.42, lr=0.000010, samples/sec=7.8, sec/step=1.026, eta=2 days, 2:55:52
2019-04-24 18:48:49 Iter 21400 [Train]: loss=62.76, epe=1.36, lr=0.000010, samples/sec=7.8, sec/step=1.024, eta=2 days, 2:49:23
2019-04-24 18:50:34 Iter 21500 [Train]: loss=70.38, epe=1.54, lr=0.000010, s

...

2019-04-25 07:46:45 Iter 47900 [Train]: loss=58.51, epe=1.24, lr=0.000010, samples/sec=4.9, sec/step=1.642, eta=2 days, 21:21:54
2019-04-25 07:50:14 Iter 48000 [Train]: loss=58.52, epe=1.23, lr=0.000010, samples/sec=4.8, sec/step=1.678, eta=2 days, 22:50:41
2019-04-25 07:50:17 Iter 48000 [Val]: loss=61.27, epe=1.45
Saving model...
... model wasn't saved -- its score (1.45) doesn't outperform other checkpoints
2019-04-25 07:53:43 Iter 48100 [Train]: loss=61.69, epe=1.31, lr=0.000010, samples/sec=5.0, sec/step=1.611, eta=2 days, 19:58:44
2019-04-25 07:57:06 Iter 48200 [Train]: loss=58.74, epe=1.25, lr=0.000010, samples/sec=4.9, sec/step=1.627, eta=2 days, 20:35:53
2019-04-25 08:00:38 Iter 48300 [Train]: loss=57.57, epe=1.21, lr=0.000010, samples/sec=4.7, sec/step=1.709, eta=3 days, 0:00:34
2019-04-25 08:04:02 Iter 48400 [Train]: loss=57.91, epe=1.21, lr=0.000010, samples/sec=4.8, sec/step=1.672, eta=2 days, 22:25:39
2019-04-25 08:07:17 Iter 48500 [Train]: loss=59.97, epe=1.27, lr=0.00001

2019-04-25 10:56:05 Iter 53500 [Train]: loss=56.99, epe=1.19, lr=0.000010, samples/sec=4.8, sec/step=1.676, eta=2 days, 20:12:02
2019-04-25 10:59:31 Iter 53600 [Train]: loss=56.96, epe=1.18, lr=0.000010, samples/sec=4.8, sec/step=1.670, eta=2 days, 19:53:55
2019-04-25 11:02:59 Iter 53700 [Train]: loss=56.55, epe=1.17, lr=0.000010, samples/sec=4.6, sec/step=1.729, eta=2 days, 22:15:06
2019-04-25 11:06:13 Iter 53800 [Train]: loss=56.09, epe=1.16, lr=0.000010, samples/sec=4.8, sec/step=1.651, eta=2 days, 19:03:35
2019-04-25 11:09:28 Iter 53900 [Train]: loss=61.16, epe=1.29, lr=0.000010, samples/sec=4.9, sec/step=1.645, eta=2 days, 18:45:28
2019-04-25 11:12:19 Iter 54000 [Train]: loss=54.50, epe=1.12, lr=0.000010, samples/sec=5.2, sec/step=1.550, eta=2 days, 14:52:47
2019-04-25 11:12:20 Iter 54000 [Val]: loss=49.97, epe=1.10
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-cyclic-sintel_finetuned/pwcnet.ckpt-54000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in 

2019-04-25 13:22:52 Iter 58900 [Train]: loss=54.33, epe=1.12, lr=0.000010, samples/sec=5.3, sec/step=1.512, eta=2 days, 11:16:32
2019-04-25 13:25:31 Iter 59000 [Train]: loss=55.87, epe=1.16, lr=0.000010, samples/sec=5.3, sec/step=1.505, eta=2 days, 10:56:13
2019-04-25 13:25:33 Iter 59000 [Val]: loss=47.28, epe=0.99
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-cyclic-sintel_finetuned/pwcnet.ckpt-59000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-cyclic-sintel_finetuned/pwcnet.ckpt-59000
2019-04-25 13:28:33 Iter 59100 [Train]: loss=57.83, epe=1.21, lr=0.000010, samples/sec=5.3, sec/step=1.496, eta=2 days, 10:33:29
2019-04-25 13:31:13 Iter 59200 [Train]: loss=54.50, epe=1.13, lr=0.000010, samples/sec=5.3, sec/step=1.504, eta=2 days, 10:50:32
2019-04-25 13:33:52 Iter 59300 [Train]: loss=57.74, epe=1.20, lr=0.000010, samples/sec=5.3, sec/step=1.511, eta=2 days, 11:04:17
2019-04-25 13:36:29 Iter 59400 [Train]: loss=55.58, epe=1.15, lr=0.0000

2019-04-25 15:52:47 Iter 64300 [Train]: loss=55.06, epe=1.15, lr=0.000010, samples/sec=5.3, sec/step=1.524, eta=2 days, 9:26:10
2019-04-25 15:55:31 Iter 64400 [Train]: loss=55.35, epe=1.14, lr=0.000010, samples/sec=5.2, sec/step=1.538, eta=2 days, 9:54:56
2019-04-25 15:58:13 Iter 64500 [Train]: loss=56.21, epe=1.17, lr=0.000010, samples/sec=5.2, sec/step=1.534, eta=2 days, 9:44:45
2019-04-25 16:00:55 Iter 64600 [Train]: loss=55.93, epe=1.16, lr=0.000010, samples/sec=5.2, sec/step=1.527, eta=2 days, 9:27:00
2019-04-25 16:03:33 Iter 64700 [Train]: loss=55.93, epe=1.17, lr=0.000010, samples/sec=5.4, sec/step=1.493, eta=2 days, 8:07:42
2019-04-25 16:06:27 Iter 64800 [Train]: loss=53.67, epe=1.11, lr=0.000010, samples/sec=5.0, sec/step=1.594, eta=2 days, 11:52:49
2019-04-25 16:09:25 Iter 64900 [Train]: loss=58.24, epe=1.23, lr=0.000010, samples/sec=4.9, sec/step=1.621, eta=2 days, 12:49:30
2019-04-25 16:12:28 Iter 65000 [Train]: loss=57.97, epe=1.23, lr=0.000010, samples/sec=4.9, sec/step=1

KeyboardInterrupt: 

## Training log

Here are the training curves for the run above:

![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/loss.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/epe.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/lr.png)

Here are the predictions issued by the model for a few validation samples:

![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val1.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val2.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val3.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val4.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val5.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val6.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val7.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val8.png)